In [2]:
import docx2txt
from book_builder.book_builder import BookBuilder

In [31]:
import sqlite3
conn = sqlite3.connect('data/kcbible.db')
db = conn.cursor()

In [28]:
book_txt = docx2txt.process('docx_normalized/Luka ei Thuthangni.docx')
lines = [line.strip() for line in book_txt.split('\n\n') if line.strip() != '']
bb = BookBuilder(lines[0])
for line in lines[1:]:
    bb.process(line)
book = bb.build()
print(f"book {book.title} has {len(book.chapters)} chapters with a combined total of {sum([len(c.verses) for c in book.chapters])} verses")

book Luka Thuthangni has 24 chapters with a combined total of 1148 verses


In [5]:
book_name = 'Luka Thuthangni'

In [29]:
def dupe_verse(c,v): return len([v1 for v1 in c.verses if v1.number == v.number]) > 1
def zero_in_verse(v): return v.number == 0

def test(c,v):
    return dupe_verse(c,v) or zero_in_verse(v)
    #return c.number == 22

def validation_errors(b):
    return [f"{c.number}:{v.number} {v.text}" for c in b.chapters for v in c.verses if test(c,v)]

validation_errors(book)

[]

In [32]:
db.execute('insert into book(name, language) values(?,?)',(book_name,'Kaang Chin'))
book_id = db.lastrowid
conn.commit()

In [33]:
insert_chapter = """
    insert into chapter(number, title, book_id)
    values(?,?,?)
    """

insert_verse = """
    insert into verse(number, title, text, chapter_id)
    values(?,?,?,?)
    """

In [34]:
for c in book.chapters:
    chapter_params = (c.number, c.title if c.title else None, book_id)
    db.execute(insert_chapter, chapter_params)
    chapter_id = db.lastrowid
    verse_params = [(v.number,v.title if v.title else None,v.text,chapter_id) for v in c.verses]
    db.executemany(insert_verse, verse_params)
conn.commit()

In [35]:
conn.close()